### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
from langchain_community.vectorstores import FAISS
import ads

##

In [ ]:
ads.set_auth(auth="resource_principal")

In [ ]:
embedings = GenerativeAIEmbeddings(
    compartment_id="ocid1.compartment.oc1..aaaaaaaah3o77etbcfg2o25jxks2pucmyrz6veg26z5lgpx3q355nikleemq",
    client_kwargs=dict(service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com") # this can be omitted after Generative AI service is GA.
)

llm = GenerativeAI(
    compartment_id="ocid1.compartment.oc1..aaaaaaaah3o77etbcfg2o25jxks2pucmyrz6veg26z5lgpx3q355nikleemq",
    client_kwargs=dict(service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com") # this can be omitted after Generative AI service is GA.
)

In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("ai-foundations.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [6]:
db = FAISS.from_documents(docs, embedings)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


In [7]:
retv = db.as_retriever()

In [8]:
chain = RetrievalQA.from_chain_type(llm=llm, retriever=retv)

In [9]:
import tempfile
from ads.llm.deploy import ChainDeployment

artifact_dir = tempfile.mkdtemp()

model = ChainDeployment(
    chain=chain,
    artifact_dir=artifact_dir,
    force_overwrite=True
)

In [10]:
model.prepare(force_overwrite=True,
        inference_conda_env="pytorch21_p39_gpu_v1",
        inference_python_version="3.9",
        )

  warnings.warn("slug will be deprecated. Provide conda pack path instead.")

INFO:ADS:To auto-extract taxonomy metadata the model must be provided. Supported models: keras, lightgbm, pytorch, sklearn, tensorflow, pyspark, and xgboost.


algorithm: null
artifact_dir:
  /tmp/tmppucwgk88:
  - - .model-ignore
    - runtime.yaml
    - score.py
    - chain.yaml
framework: null
model_deployment_id: null
model_id: null

In [11]:
model.verify("what is AI Foundations course?")

{'query': 'what is AI Foundations course?',
 'result': ' The Oracle Cloud Infrastructure AI Foundations course is an introduction to AI and Machine Learning (ML). The course is intended for anyone who wants to begin learning about AI and ML, such as cloud engineers or architects, IT professionals, or students seeking to upskill. The course is split into six modules and is recommended for those who are familiar with OCI or similar cloud services. \n\nWould you like me to go into more detail about any specific parts of the course? '}

In [12]:
model.save()

['.model-ignore', 'runtime.yaml', 'score.py', 'chain.yaml']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

'ocid1.datasciencemodel.oc1.us-chicago-1.amaaaaaajcisnjiaiqnnnoj7dbzxdgeqddh5efevanpira6kmy3lse2ahsna'

In [13]:
model.deploy(environment_variables={"CRYPTOGRAPHY_OPENSSL_NO_LEGACY":"1"}, 
             deployment_log_group_id="ocid1.loggroup.oc1.us-chicago-1.amaaaaaajcisnjiazonw73ce54gyv2i32x7rgx42zlr65h2sbzgjobipt5mq",
             deployment_access_log_id="ocid1.log.oc1.us-chicago-1.amaaaaaajcisnjiakhi42opuay45virks2b4omdhsvxxx2xnenvkegwjz3pa")

Model Deployment OCID: ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaajcisnjiaysxsjkax3a2dm4oypuner72ha5h6mweb7cgpmhea73ja


Creating model deployment:   0%|          | [00:00<?, ?it/s]


kind: deployment
spec:
  createdBy: ocid1.datasciencenotebooksession.oc1.us-chicago-1.amaaaaaajcisnjia2j7o66wzobthxu2byl6hsdnw2m7e7rtazu3aovgsbora
  definedTags:
    Oracle-Tags:
      CreatedBy: ocid1.datasciencenotebooksession.oc1.us-chicago-1.amaaaaaajcisnjia2j7o66wzobthxu2byl6hsdnw2m7e7rtazu3aovgsbora
      CreatedOn: '2024-05-14T16:53:01.459Z'
  displayName: attractive-ant-2024-05-14-16:53.01
  id: ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaajcisnjiaysxsjkax3a2dm4oypuner72ha5h6mweb7cgpmhea73ja
  infrastructure:
    kind: infrastructure
    spec:
      accessLog:
        logGroupId: ocid1.loggroup.oc1.us-chicago-1.amaaaaaajcisnjiazonw73ce54gyv2i32x7rgx42zlr65h2sbzgjobipt5mq
        logId: ocid1.log.oc1.us-chicago-1.amaaaaaajcisnjiakhi42opuay45virks2b4omdhsvxxx2xnenvkegwjz3pa
      bandwidthMbps: 10
      compartmentId: ocid1.compartment.oc1..aaaaaaaah3o77etbcfg2o25jxks2pucmyrz6veg26z5lgpx3q355nikleemq
      deploymentType: SINGLE_MODEL
      policyType: FIXED_SIZE
  

In [14]:
model.predict("Who are instructors for AI Foundations?")

{'query': 'Who are instructors for AI Foundations?',
 'result': ' The instructors for the Oracle Cloud Infrastructure AI Foundations course are Hemant Gahankari, Himanshu Raj, and Nick Commisso. \n\nWould you like to know more about the course or its instructors? '}